# IBM Data Science Specialisation
### Capstone Project
This is the primary notebook for completing the capstone project for the IBM Data Science Specialisation on Coursera.com

# 1.0 Introduction

### Business problem: Euro-trip
You're a big city person planning your next trip around Europe. You know that you will only have time to visit 3 countries and during previous travels you've discovered that you LOVE Paris. 
The plan is therefore to find european capitols with a similar feel to Paris.


# 2.0 Data
To compare the capitals we make the following assumptions:
1) The similarity of cities can be estimated based on a venue category profile

2) A sufficient venue category profile can be derived from the 100 nearest venues to the capitols official coordinates

To gather the data we need to create these venue category profiles we:
1) Identify a webpage from where we can scrape the names of the capitals

2) Scrape the capital names from the given site

3) Determine the coordinates of each capital

4) Get venue data for each capital

Scraping will be done with BeautifulSoup.
Location data will be found with Nominatim.
Venue data will be found with the FourSquare API.

## 2.1 Load packages
The following packages are imported for data gathering and analysis:
- Pandas: Data handling

- Numpy: Math

- bs4: Scraping

- Geopy: Location data

- Folium: Maps


In [1]:
# For data handling
import pandas as pd

# For math
import numpy as np

# For scraping
import requests
from bs4 import BeautifulSoup

# For visualisation
import matplotlib.cm as cm
import matplotlib.colors as colors

try:
    import folium
except:
    !pip install folium
    import folium
    
# For location data
try:
    from geopy.geocoders import Nominatim
except:
    !pip install geopy
    from geopy.geocoders import Nominatim
    

## 2.2 Scraping
To get an easy list of european capitals we use the website "www.nationsonline.org".
We define the url and use bs4 to read the underlying html.

In [2]:
url = 'https://www.nationsonline.org/oneworld/capitals_europe.htm'
page = requests.get(url)
soup = BeautifulSoup(page.content,'html5lib')

We then scrape the html code for the relevant data. 
In this case it was in a table and was easiest to isolate through the map link column.
The data is put into a pandas dataframe and sorted alphabetically for user friendliness.

In [3]:
table_content = []
table = soup.find('table')

for row in table.findAll('td'):
    cell = {}
    
    if row.img:
        cell['Capitol'] = row.text.split('Map')[0].replace('\n', ' ')
        table_content.append(cell)

df = pd.DataFrame(table_content)
df = df.sort_values('Capitol',ascending = True).reset_index(drop=True)
df.head()

,Capitol
0,Amsterdam
1,Andorra la Vella
2,Athens
3,Belgrade
4,Berlin


## 2.3 Location data
We start by adding two columns to our dataframe - 'Latitude' and 'Longitude'.
Then we use Nominatim to obtain location data for each capitol in the dataframe.

In [4]:
df['Latitude'] = np.nan
df['Longitude'] = np.nan


geolocator = Nominatim(user_agent = "Locator")

for index, row in df.iterrows():
    locstring = row['Capitol']
    location = geolocator.geocode(locstring)
    
    try:
        #print('Capitol: {} \n Lat: {} \n Lng: {}'.format(row['Capitol'],location.latitude,location.longitude))
        df['Latitude'][index] = location.latitude
        df['Longitude'][index] = location.longitude
    except:
        #print('Didnt work for capitol:',row['Capitol'])
        pass
    
df.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Capitol,Latitude,Longitude
0,Amsterdam,52.372760,4.893604
1,Andorra la Vella,42.506939,1.521247
2,Athens,37.983941,23.728305
3,Belgrade,44.817813,20.456897
4,Berlin,52.517037,13.388860


In order to check the validity of the location data, a quick folium map is created to illustrate the capitals.

In [5]:
# Create map
eu_loc = geolocator.geocode('Europe')
eu_map = folium.Map(location=[eu_loc.latitude, eu_loc.longitude], zoom_start=4)

# add markers to map
for lat, lng, capitol in zip(df['Latitude'], df['Longitude'], df['Capitol']):
    label = '{}'.format(capitol)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(eu_map)  
    
eu_map

Most data seems correct with the exception of "Sofia" which found "Sofia, Madagascar" instead of the "Sofia, Bulgaria".
It is decided to drop this entry.

In [6]:
df = df.drop(df[df['Capitol'].str.contains("Sofia", na=False, case=False)].index)
df.reset_index(drop=True, inplace=True)

## 2.4 Venue data
To get venue data we first need to specify our Foursquare API credentials

In [7]:
# Foursquare API credentials
CLIENT_ID = 'LXHUDUBYOD5TK2VVROTXSP2H0HDR1LLUOSMRYHI4GLP02HNA' # your Foursquare ID
CLIENT_SECRET = 'F0NK2IS1SGGKNCAWZNQFR5RF0JF0UODTRKJD5F3ZFA01CD1G' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

ACCESS_TOKEN = 'EG0BTYIVHTLOG4BO04STPG4RBG44ZL4MS022M4B2SXWRJYEA' # your FourSquare Access Token

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LXHUDUBYOD5TK2VVROTXSP2H0HDR1LLUOSMRYHI4GLP02HNA
CLIENT_SECRET:F0NK2IS1SGGKNCAWZNQFR5RF0JF0UODTRKJD5F3ZFA01CD1G


For simplicity, we utilize the function for getting nearby venues from the lab.

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius, limit):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print('Capitol: {} \n Lat: {}, Lng: {} \n Radius: {}, Limit: {}'.format(name, lat, lng, radius, limit))

        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                    CLIENT_ID, 
                    CLIENT_SECRET, 
                    VERSION, 
                    lat, 
                    lng, 
                    radius, 
                    limit)

        results = requests.get(url).json()["response"]['groups'][0]['items']

        venues_list.append([(
                    name, 
                    lat, 
                    lng, 
                    v['venue']['name'], 
                    v['venue']['location']['lat'], 
                    v['venue']['location']['lng'],  
                    v['venue']['categories'][0]['name']) for v in results])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Capitol', 
                          'Capitol Latitude', 
                          'Capitol Longitude', 
                          'Venue', 
                          'Venue Latitude', 
                          'Venue Longitude', 
                          'Venue Category']

    return(nearby_venues)

We use the function getNearbyVenues to create our venue dataframe - eu_venues:

In [9]:
eu_venues = getNearbyVenues(df['Capitol'],df['Latitude'],df['Longitude'],1000,200)

Capitol:  Amsterdam  
 Lat: 52.3727598, Lng: 4.8936041 
 Radius: 1000, Limit: 200
Capitol:  Andorra la Vella  
 Lat: 42.5069391, Lng: 1.5212467 
 Radius: 1000, Limit: 200
Capitol:  Athens  
 Lat: 37.9839412, Lng: 23.7283052 
 Radius: 1000, Limit: 200
Capitol:  Belgrade  
 Lat: 44.8178131, Lng: 20.4568974 
 Radius: 1000, Limit: 200
Capitol:  Berlin  
 Lat: 52.5170365, Lng: 13.3888599 
 Radius: 1000, Limit: 200
Capitol:  Bern  
 Lat: 46.9482713, Lng: 7.4514512 
 Radius: 1000, Limit: 200
Capitol:  Bratislava  
 Lat: 48.1516988, Lng: 17.1093063 
 Radius: 1000, Limit: 200
Capitol:  Brussels  
 Lat: 50.8465573, Lng: 4.351697 
 Radius: 1000, Limit: 200
Capitol:  Bucharest  
 Lat: 44.4361414, Lng: 26.1027202 
 Radius: 1000, Limit: 200
Capitol:  Budapest  
 Lat: 47.4983815, Lng: 19.0404707 
 Radius: 1000, Limit: 200
Capitol:  Chisinau  
 Lat: 47.0245117, Lng: 28.8322923 
 Radius: 1000, Limit: 200
Capitol:  Copenhagen  
 Lat: 55.6867243, Lng: 12.5700724 
 Radius: 1000, Limit: 200
Capitol:  Dubli

In [10]:
eu_venues.head()

,Capitol,Capitol Latitude,Capitol Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Amsterdam,52.37276,4.893604,Proeflokaal De Drie Fleschjes,52.374203,4.892239,Bar
1,Amsterdam,52.37276,4.893604,Scheltema,52.372205,4.893175,Bookstore
2,Amsterdam,52.37276,4.893604,Wynand Fockink,52.372301,4.895253,Liquor Store
3,Amsterdam,52.37276,4.893604,Hotel V,52.371125,4.893665,Hotel
4,Amsterdam,52.37276,4.893604,Sofitel Legend The Grand Amsterdam,52.371093,4.895410,Hotel


## 2.5 Quick analysis of data
To get a feel for the venue data, we run a quick analysis on the dataframe to check:
- Shape

- Venues per capitol

- Maximum number of venues in a capitol

- Minimum number of venues in a capitol

- Amount of unique venue categories 

In [11]:
print('Shape:',eu_venues.shape,'\n')

print(eu_venues.describe(include='all'))


Shape: (4120, 7) 

           Capitol  Capitol Latitude  Capitol Longitude      Venue  \
count         4120       4120.000000        4120.000000       4120   
unique          45               NaN                NaN       3992   
top      Belgrade                NaN                NaN  Starbucks   
freq           100               NaN                NaN         13   
mean           NaN         48.695997          14.350967        NaN   
std            NaN          6.954247          12.184132        NaN   
min            NaN         35.173930         -51.735539        NaN   
25%            NaN         42.506939           7.424224        NaN   
50%            NaN         48.208354          15.977177        NaN   
75%            NaN         53.349764          23.728305        NaN   
max            NaN         64.175029          37.617494        NaN   

        Venue Latitude  Venue Longitude Venue Category  
count      4120.000000      4120.000000           4120  
unique             NaN    

The statistics overview shows us that:
1) There are 4120 entries in the dataframe

2) There are 45 unique capitols

3) There are 3992 unique venue names - i.e. some venues have the same name, e.g. Starbucks.

4) There are 338 unique venue categories

5) The most common venue category across the dataframe is "Hotel" with a frequency of 246.


Some capitols may not have enough venues to make a suitable venue category profile. If we assume that the limit is 25 venues, let us investigate which capitols to drop:

In [12]:
eu_venues_count = eu_venues[['Capitol','Venue']].groupby('Capitol').count()

eu_venues_count[eu_venues_count['Venue'] < 25]


,Venue
Capitol,
Nuuk,4
San Marino,11


As you can see, based on our assumption we should drop Nuuk and San Marino.

In [13]:
df = df.drop(df[df['Capitol'].str.contains("Nuuk", na=False, case=False)].index)
df = df.drop(df[df['Capitol'].str.contains("San Marino", na=False, case=False)].index)
df.reset_index(drop=True, inplace=True)

With a clean dataframe we are now ready to start working on our clustering method.



# 3.0 Method


## 3.1 Load packages
The following packages are imported for data gathering and analysis:
- sklearn: Clustering algorithm


In [14]:
# For clustering
from sklearn.cluster import KMeans


## 3.2 Feature extraction and pre-processing
To create our venue category profile we will:
1) One hot encode our venue categories to make them countable and comparable

2) Group the one hot encoded venue categories for each capitol with a mean aggregate function to obtain the distribution of venue categories, i.e. the venue category profile.


In [15]:
# one hot encoding
eu_onehot = pd.get_dummies(eu_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
eu_onehot['Capitol'] = eu_venues['Capitol'] 

# move neighborhood column to the first column
fixed_columns = [eu_onehot.columns[-1]] + list(eu_onehot.columns[:-1])
eu_onehot = eu_onehot[fixed_columns]


In [16]:
eu_grouped = eu_onehot.groupby('Capitol').mean().reset_index()
eu_grouped.head()

,Capitol,Accessories Store,Adult Boutique,African Restaurant,Alsatian Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Armenian Restaurant,Art Gallery,...,Watch Shop,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Amsterdam,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,...,0.0,0.00,0.010000,0.000000,0.000000,0.0,0.01,0.0,0.0,0.0
1,Andorra la Vella,0.0,0.0,0.0,0.0,0.010638,0.0,0.0,0.0,0.00,...,0.0,0.00,0.000000,0.010638,0.000000,0.0,0.00,0.0,0.0,0.0
2,Athens,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,...,0.0,0.00,0.013333,0.053333,0.013333,0.0,0.00,0.0,0.0,0.0
3,Belgrade,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.02,...,0.0,0.00,0.000000,0.010000,0.000000,0.0,0.00,0.0,0.0,0.0
4,Berlin,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.05,...,0.0,0.01,0.000000,0.020000,0.000000,0.0,0.00,0.0,0.0,0.0


## 3.3 Clustering 
To determine similar cities we use a clustering method called k-means. This considers the problem geometric and solves the problem by:
1) Initializing "k" amount of cluster centroids as geometric positions within the feature space 

2) Optimizes the position of the cluster centroids by minimizing the sum of "euclidian distances" from each feature to its closest cluster centroid 

For the purpose of this investigation we assume that dividing the european capitols into 8 clusters should give a sufficient result.

In [98]:
# set number of clusters
kclusters = 6

eu_grouped_clustering = eu_grouped.drop('Capitol', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(eu_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 4, 3, 0, 4, 0, 0, 0, 3], dtype=int32)

## 3.4 Visualizations

To visualize the results we first need to add labels to the clusters. To do so, we utilize the return_most_common_venues function from the labs and sort by the top 5 venues for each capitol.

In [99]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [100]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Capitol']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
capitol_venues_sorted = pd.DataFrame(columns=columns)
capitol_venues_sorted['Capitol'] = eu_grouped['Capitol']

for ind in np.arange(eu_grouped.shape[0]):
    capitol_venues_sorted.iloc[ind, 1:] = return_most_common_venues(eu_grouped.iloc[ind, :], num_top_venues)

capitol_venues_sorted.head()

,Capitol,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Amsterdam,Hotel,Bar,Bookstore
1,Andorra la Vella,Restaurant,Hotel,Spanish Restaurant
2,Athens,Dessert Shop,Café,Coffee Shop
3,Belgrade,Hotel,Coffee Shop,Restaurant
4,Berlin,Hotel,History Museum,Art Gallery


In [101]:
# add clustering labels

try:
    capitol_venues_sorted.pop('Cluster Labels')
    capitol_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
    
except:
    capitol_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
    
eu_merged = df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
eu_merged = eu_merged.join(capitol_venues_sorted.set_index('Capitol'), on='Capitol')

eu_merged.head() # check the last columns!

,Capitol,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Amsterdam,52.372760,4.893604,3,Hotel,Bar,Bookstore
1,Andorra la Vella,42.506939,1.521247,3,Restaurant,Hotel,Spanish Restaurant
2,Athens,37.983941,23.728305,4,Dessert Shop,Café,Coffee Shop
3,Belgrade,44.817813,20.456897,3,Hotel,Coffee Shop,Restaurant
4,Berlin,52.517037,13.388860,0,Hotel,History Museum,Art Gallery


With the cluster labels in place, we can now present a map of our findings:

In [102]:
# create map
eu_loc = geolocator.geocode('Europe')
eu_map_clusters = folium.Map(location=[eu_loc.latitude, eu_loc.longitude], zoom_start=4)

eu_merged['Cluster Labels'].astype('int32')

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(eu_merged['Latitude'], eu_merged['Longitude'], eu_merged['Capitol'], eu_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    cluster = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(eu_map_clusters)
       
eu_map_clusters

Lets look further into the Paris cluster

In [157]:
ParisCluster = eu_merged[eu_merged['Capitol'].str.contains("Paris",na=False,case=False)]['Cluster Labels']

eu_ParisCluster = eu_merged[eu_merged['Cluster Labels'] == int(ParisCluster)]
print(eu_ParisCluster['Capitol'])
print(eu_ParisCluster[['Capitol','1st Most Common Venue']].groupby('1st Most Common Venue').count().sort_values('Capitol',ascending=False))
print(eu_ParisCluster[['Capitol','2nd Most Common Venue']].groupby('2nd Most Common Venue').count().sort_values('Capitol',ascending=False))
print(eu_ParisCluster[['Capitol','3rd Most Common Venue']].groupby('3rd Most Common Venue').count().sort_values('Capitol',ascending=False))

4               Berlin 
6           Bratislava 
7             Brussels 
8            Bucharest 
10            Chisinau 
11          Copenhagen 
13            Helsinki 
14                Kiev 
17              London 
18     Luxembourg City 
20               Minsk 
22              Moscow 
24                Oslo 
25               Paris 
27              Prague 
34             Tallinn 
40             Vilnius 
41              Warsaw 
Name: Capitol, dtype: object
                       Capitol
1st Most Common Venue         
Coffee Shop                  6
Hotel                        3
Café                         2
French Restaurant            2
Plaza                        2
Chocolate Shop               1
Cocktail Bar                 1
Park                         1
                       Capitol
2nd Most Common Venue         
Coffee Shop                  6
Hotel                        5
Café                         2
Bakery                       1
Bar                          1
Bookstore   

# 4.0 Results
The clustering derived throughout this report shows that Paris has a venue category profile heavily based on coffee shops, hotels and cafés.

Similar european capitols include:
- Berlin 
- Bratislava 
- Brussels 
- Bucharest 
- Chisinau 
- Copenhagen 
- Helsinki 
- Kiev 
- London 
- Luxembourg City 
- Minsk 
- Moscow 
- Oslo 
- Prague 
- Tallinn 
- Vilnius 
- Warsaw 

# 5.0 Discussion
Throughout this investigation, the following observations were made:
1) It could be interesting to evolve the dataset to include country and/or capitol economic and demographic data

2) It could be interesting to include more venues per capitol when possible. With a limitation of 100 venues the profile is very limited.

3) It could be interesting to group venue categories when possible - e.g. Group restaurants, group cafe/coffee shop, etc.

4) It could be interesting to extract other features from the data set - e.g. amount of venues, venue distribution statistics, etc.


# 6.0 Conclusion
Based on the results from this investigation, these are the recommendations for the euro-trip:
- If you want to stay close to Paris, visit London, Luxembourg City and Brussels

- If you want to go further stay in nothern and eastern Europe



